In [ ]:
import pandas as pd
import sqlalchemy
def create_df(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:, ['s','E','p']]
    df.columns = ['symbol', 'Time','Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df
engine = sqlalchemy.create_engine('sqlite:///ETHUSDTstream.db')

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from binance import AsyncClient, BinanceSocketManager


async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ts = bm.trade_socket('ETHUSDT')
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            df = create_df(res)
            df.to_sql('ETHUSDT', engine, if_exists='append', index=False)

    await client.close_connection()

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())